http://tdual.hatenablog.com/entry/2018/04/09/133000

LDAのコードを実装
TODO::可視化のところの実装
前のデータからちゃんとtagを持ってくる（突貫工事でlangを持ってきたものの、件数(種類)が多すぎるのかうまく表示できず）


現在280件のデータ(半月分)からスコープを作成している


In [3]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
import re
import MeCab
import random
from gensim import corpora, models

In [4]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/English.txt")
stopwords += [line.decode("utf-8").strip() for line in res]

In [5]:
len(stopwords)

910

In [6]:
class Tokenizer:
    def __init__(self, stopwords, parser=None, include_pos=None, exclude_posdetail=None, exclude_reg=None):
    
        self.stopwords = stopwords
        self.include_pos = include_pos if include_pos else  ["名詞", "動詞", "形容詞"]
        self.exclude_posdetail = exclude_posdetail if exclude_posdetail else ["接尾", "数"]
        self.exclude_reg = exclude_reg if exclude_reg else r"$^"  # no matching reg
        if parser:
            self.parser = parser
        else:
            mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")
            self.parser = mecab.parse
            

    def tokenize(self, text, show_pos=False):
        text = re.sub(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+", "", text)    #URL
        text = re.sub(r"\"?([-a-zA-Z0-9.`?{}]+\.jp)\"?" ,"", text)  # xxx.jp 
        text = text.lower()
        l = [line.split("\t") for line in self.parser(text).split("\n")]
        res = [
            i[2] if not show_pos else (i[2],i[3]) for i in l 
                if len(i) >=4 # has POS.
                    and i[3].split("-")[0] in self.include_pos
                    and i[3].split("-")[1] not in self.exclude_posdetail
                    and not re.search(r"(-|−)\d", i[2])
                    and not re.search(self.exclude_reg, i[2])
                    and i[2] not in self.stopwords          
            ]
        return res



In [7]:
import pandas as pd
df_content = pd.read_csv('data/2018-07-15-content.csv')
df_content.head(3)
df_nonna = df_content[df_content['text'].isna() == False]
df_nonna = df_nonna[df_nonna['lang'].isna() == False]

In [8]:
df_content.head(3)

,Unnamed: 0,id,text,code-frame,lang,code,a-text,a-href
0,0,1a7920d0dbc2099db000,はじめには、モックのサービスを作成して、実際にシナリオテストを実施してみました。今回は、テス...,Scenario: Normal def\n\n* def text = 'karate e...,"cucumber,cucumber,cucumber,cucumber,cucumber,c...","def,print,match,"""""",jsonData,set,table,set,tab...","前回（#03 Writing Scenarios）,https://github.com/i...","#%E3%81%AF%E3%81%98%E3%82%81%E3%81%AB,https://..."
1,1,5414e0beb7df9908949d,久々のQiita投稿です。淫夢記事しか投稿してないのはまずいかなと思ってまともな記事を1つシ...,\n# # notepadの位置を確認\n$ which notepad.exe\n/mnt...,"sh,sh",NaN,WSLのbashからWindowsの実行ファイルを実行する | yuipro,https://yuis-programming.com/2018/06/19/wsl%e3...
2,2,79679ad9911da6ea0309,私は現在、Scalaで書かれているわりと古めのゲーム用サーバーに、いろいろ機能追加したり不具...,"command: findAndModify { findandmodify: ""chara...","js,text","var,findAndModify,findAndModify,findAndModify,...","Goでテストクライアントを書いたり,1,Goでテストクライアントを書くことで、安価なLinu...",https://qiita.com/yonezawaizumi/items/01384aad...


In [9]:
type(df_content['text'][0])

str

In [10]:
s = 'aaa@xxx.com bbb@yyy.com ccc@zzz.com'
type(s)
re.sub(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+", "", s)    #URL

'aaa@xxx.com bbb@yyy.com ccc@zzz.com'

In [21]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")
t = Tokenizer(stopwords + ["…。"] , mecab.parse, exclude_reg=r"\d(年|月|日)")


docs = []


for texts in df_nonna['text']:
#     if(len(docs) > 50):
#         break
    docs.append(t.tokenize(texts))
    
    
# for a in articles:
#     with a.open() as f:
#         f.readline()
#         f.readline()
#         docs.append(tokenize(f.read()))

In [22]:
d = corpora.Dictionary(docs)

adding document #0 to Dictionary(0 unique tokens: [])
built Dictionary(33191 unique tokens: ['API', 'CLASSPATH', 'JSON', 'demo', 'feature']...) from 2790 documents (total 642178 corpus positions)


In [23]:
print(d[1000])
print(d.token2id['謎'])

謎
1000


使われている文書の数がno_belowより少ない単語を無視し、no_aboveの割合以上の文書に出てくる単語を無視しています。
また、compactifyでIDを振り直してコンパクトにする

In [24]:
d.filter_extremes(no_below=5, no_above=0.2)
d.compactify()

discarding 26724 tokens: [('karate', 1), ('ある', 1829), ('いる', 2090), ('おく', 805), ('くださる', 689), ('する', 2648), ('できる', 1790), ('なる', 1816), ('の', 1620), ('みる', 1241)]...
keeping 6467 tokens which were in no less than 5 and no more than 558 (=20.0%) documents
resulting dictionary: Dictionary(6467 unique tokens: ['API', 'CLASSPATH', 'JSON', 'demo', 'feature']...)


In [25]:
corpus = [d.doc2bow(w) for w in docs]
test_size = int(len(corpus) * 0.1)
test_corpus = corpus[:test_size]
train_corpus = corpus[test_size:]

In [26]:
logging.basicConfig(format='%(message)s', level=logging.INFO)

## 学習させる


In [27]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=d, num_topics=50, passes=10)

using symmetric alpha at 0.02
using symmetric eta at 0.02
using serial LDA version on this node
running online (multi-pass) LDA training, 50 topics, 10 passes over the supplied corpus of 2790 documents, updating model once every 2000 documents, evaluating perplexity every 2790 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/2790
merging changes from 2000 documents into a model of 2790 documents
topic #1 (0.020): 0.010*"クラス" + 0.010*"関数" + 0.008*"モデル" + 0.006*"実装" + 0.005*"変数" + 0.005*"プログラム" + 0.005*"画面" + 0.004*"引数" + 0.004*"予測" + 0.004*"部分"
topic #17 (0.020): 0.021*"要素" + 0.013*"疎い" + 0.007*"配列" + 0.007*"選択" + 0.007*"値" + 0.006*"実装" + 0.005*"状態" + 0.005*"undefined" + 0.005*"命令" + 0.004*"UNITY"
topic #4 (0.020): 0.010*"関数" + 0.008*"定義" + 0.007*"出力" + 0.005*"テスト" + 0.005*"要素" + 0.004*"引数" + 0.004*"良い" + 0.004*"値" + 0.004*"対応" + 0.004*"参照"
topic #28 (0.020): 0.016*"命令" + 0.010*"レジスタ" + 0.008*"bit" + 0.007*"クラス" + 0.006*"小数点" + 0.006*

PROGRESS: pass 4, at document #2000/2790
merging changes from 2000 documents into a model of 2790 documents
topic #0 (0.020): 0.095*"モジュール" + 0.035*"CSV" + 0.021*"信号" + 0.019*"出力" + 0.018*"エクセル" + 0.016*"リ" + 0.016*"ダウンロード" + 0.015*"スクリプト" + 0.014*"入出力" + 0.013*"入力"
topic #28 (0.020): 0.073*"命令" + 0.042*"レジスタ" + 0.033*"bit" + 0.027*"オペランド" + 0.021*"小数点" + 0.020*"浮動" + 0.018*"例外" + 0.014*"メモリ" + 0.014*"整数" + 0.013*"パック"
topic #17 (0.020): 0.125*"要素" + 0.054*"配列" + 0.025*"疎い" + 0.022*"インデックス" + 0.018*"DB2" + 0.016*"width" + 0.016*"幅" + 0.014*"undefined" + 0.014*"高さ" + 0.013*"ジョブ"
topic #39 (0.020): 0.022*"AWS" + 0.015*"起動" + 0.014*"ユーザー" + 0.013*"db" + 0.011*"アクセス" + 0.011*"サービス" + 0.009*"権限" + 0.009*"コンテナ" + 0.008*"ログイン" + 0.008*"iam"
topic #16 (0.020): 0.041*"問題" + 0.022*"キャッシュ" + 0.021*"特徴" + 0.015*"パフォーマンス" + 0.014*"コンテスト" + 0.014*"有効" + 0.013*"AtCoder" + 0.012*"プログラム" + 0.012*"値" + 0.011*"期限"
topic diff=1.463144, rho=0.395439
-7.699 per-word bound, 207.8 perplexity estimate based on

topic #12 (0.020): 0.019*"画面" + 0.016*"更新" + 0.016*"項目" + 0.016*"ユーザ" + 0.015*"id" + 0.015*"テーブル" + 0.014*"メール" + 0.014*"タグ" + 0.013*"キー" + 0.012*"情報"
topic #0 (0.020): 0.103*"モジュール" + 0.041*"CSV" + 0.028*"出力" + 0.020*"信号" + 0.019*"入力" + 0.018*"ダウンロード" + 0.018*"エクセル" + 0.016*"リ" + 0.015*"入出力" + 0.014*"表"
topic #34 (0.020): 0.016*"CSS" + 0.013*"ライブラリ" + 0.012*"スタイル" + 0.011*"JS" + 0.008*"リスト" + 0.008*"graphql" + 0.007*"FRAMeWORK" + 0.007*"使える" + 0.007*"持つ" + 0.007*"デバッグ"
topic #15 (0.020): 0.026*"プロジェクト" + 0.017*"ビルド" + 0.016*"開発" + 0.014*"アプリ" + 0.012*"バージョン" + 0.011*"起動" + 0.011*"アプリケーション" + 0.009*"選択" + 0.009*"windows" + 0.009*"開く"
topic diff=0.353918, rho=0.310161
-7.521 per-word bound, 183.7 perplexity estimate based on a held-out corpus of 790 documents with 105477 words
PROGRESS: pass 8, at document #2790/2790
merging changes from 790 documents into a model of 2790 documents
topic #47 (0.020): 0.037*"実装" + 0.014*"画面" + 0.010*"座標" + 0.008*"入力" + 0.007*"ボタン" + 0.007*"コントローラ" + 0.00

In [28]:
N = sum(count for doc in test_corpus for id, count in doc)
print("N: ",N)

perplexity = np.exp2(-lda.log_perplexity(test_corpus))
print("perplexity:", perplexity)

N:  42112


-10.845 per-word bound, 1839.6 perplexity estimate based on a held-out corpus of 279 documents with 42112 words


perplexity: 1839.6103603724669


20 っこのトピックを表示してみる

In [29]:
def get_topic_words(topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))
        
        
for t in range(20):
    print("Topic # ",t)
    get_topic_words(t)
    print("\n")

Topic #  0
モジュール: 0.11059997230768204
CSV: 0.043365977704524994
出力: 0.02912191115319729
エクセル: 0.02114671841263771
ダウンロード: 0.01827458292245865
入力: 0.018209226429462433
信号: 0.01673273555934429
EXCEL: 0.013431021012365818
スクリプト: 0.013326830230653286
入出力: 0.012866916134953499


Topic #  1
モデル: 0.019530732184648514
解析: 0.01201239787042141
定義: 0.010616951622068882
関数: 0.010229391045868397
VIEW: 0.008985454216599464
分析: 0.008905107155442238
考える: 0.008560089394450188
良い: 0.008244260214269161
情報: 0.007540079765021801
言う: 0.006542948540300131


Topic #  2
Re・Act: 0.07579191029071808
angular: 0.04981851950287819
native: 0.03881143033504486
THREAD: 0.0303866658359766
SAM: 0.022575009614229202
アプリ: 0.01740042306482792
サンプル: 0.016208035871386528
redux: 0.0161623302847147
コンポーネント: 0.015436164103448391
JavaScript: 0.014651309698820114


Topic #  3
温度: 0.04743185639381409
frac: 0.04589875787496567
case: 0.033589571714401245
partial: 0.03280976042151451
定義: 0.03016696497797966
int: 0.02486519142985344
関

In [30]:
topics = sorted(lda.get_document_topics(corpus[0]), key=lambda t:t[1], reverse=True)
for t in topics[:10]:
    print("{}: {}".format(t[0], t[1]))

4: 0.47234076261520386
41: 0.13297370076179504
22: 0.1090388149023056
31: 0.08123576641082764
45: 0.07855966687202454
48: 0.05498792976140976
12: 0.01990543305873871
24: 0.019325951114296913
46: 0.015223784372210503
37: 0.01134180836379528


In [31]:
for t in topics[:10]:
    print("Topic # ",t[0])
    get_topic_words(t[0])
    print("\n")

Topic #  4
出力: 0.028276966884732246
定義: 0.01403904426842928
引数: 0.011552038602530956
渡す: 0.011542421765625477
くれる: 0.0109279565513134
言語: 0.010195373557507992
記述: 0.009658120572566986
良い: 0.008640352636575699
ん: 0.008033470250666142
関数: 0.007946304976940155


Topic #  41
関数: 0.0382821150124073
値: 0.014656711369752884
リクエスト: 0.014092267490923405
定義: 0.011392643675208092
変数: 0.009932107292115688
JavaScript: 0.009777076542377472
代入: 0.009098599664866924
スクリプト: 0.008686017245054245
返す: 0.00845917034894228
記述: 0.007835714146494865


Topic #  22
テスト: 0.14645443856716156
自動: 0.01541009359061718
実装: 0.014386479742825031
メソッド: 0.013915706425905228
対象: 0.011809773743152618
生成: 0.01062107179313898
クラス: 0.009682255797088146
CI: 0.00904933176934719
Clova: 0.008943438529968262
定義: 0.00805339403450489


Topic #  31
python: 0.13295336067676544
TensorFlow: 0.02631799504160881
プラグイン: 0.02288876101374626
JSON: 0.0214704442769289
pyenv: 0.02021012455224991
py: 0.017710035666823387
バージョン: 0.015101728960871

In [32]:
df_nonna['text'][0]

'はじめには、モックのサービスを作成して、実際にシナリオテストを実施してみました。今回は、テストの幅を広げるために、Karateで、どのようにデータの定義をするのかを確認してみます。テスト実施時に、リクエストの内容を指定したり、レスポンスの内容を確認したりするために、どのようにデータを定義するのか、ということになります。データの指定方法変数の指定まずは、基本的な定義として、変数での指定をしてみます。が、変数の定義です。 は、テスト実行時に、レポートに変数の内容を出力してくれます。 は、値の確認です。実行したときのレポートの内容は、以下のような感じです。問題なく、テストが通っていますね。当然ながら、日本語も問題なく利用できます。JSON形式のデータの指定次は、JSONデータの指定方法について確認します。REST-APIのテストに向けては、この辺りの使い勝手は重要ですね。基本的な指定の仕方として、マルチラインでの指定です。 で囲むことによって、マルチラインでのデータの指定ができています。先程指定した  のデータの一部を変更しています。コマンドを利用することで、データを変更できます。今度は、テーブル形式でのJSONデータの指定です。 コマンドを指定して定義しています。生のJSONデータよりも、こちらの方が見やすいですね。ただ、tableの場合、ネストされているようなJSONは定義できません。ネストされたJSONデータを定義したい場合は、コマンドを利用することで可能です。こちらも、テーブル形式に近い内容ですが、のときと比べて、行と列の指定が入れ替わっていることに注意してください。テストの実行結果は、以下のようになります。指定したデータの内容も確認できて、分かりやすいです。パラメータの置換上記で、コマンドを利用した変数の値の変換を実施しましたが、指定した変数の一部を置き換えるようなこと（例えば、文字列の一部分を変更した場合など）はできません。そのような置換としては、別の方法があります。コマンドを利用すると、予め置換変数を指定しておき、それを指定した値で置換するようなことが可能です。デフォルトの変数は、 という形式です。この場合、` のようなかたちで、変数の一部を置換できます。置換パラメータは、形式をカスタマイズすることも可能です。 のように定義して、置換することも可能です。こ

In [33]:
all_topics = lda.get_document_topics(corpus, minimum_probability=0)
# <||
# 試しに0番目の文書のトピックを見ましょう。
# >|python|
print(all_topics[0])

[(0, 0.00012665863), (1, 0.00012665863), (2, 0.00012665863), (3, 0.00012665863), (4, 0.4690496), (5, 0.00012665863), (6, 0.00012665863), (7, 0.00012665863), (8, 0.00012665863), (9, 0.00012665863), (10, 0.00012665863), (11, 0.00012665863), (12, 0.00012665863), (13, 0.00012665863), (14, 0.00012665863), (15, 0.00012665863), (16, 0.00012665863), (17, 0.00012665863), (18, 0.00012665863), (19, 0.00012665863), (20, 0.00012665863), (21, 0.00012665863), (22, 0.109085724), (23, 0.00012665863), (24, 0.019816684), (25, 0.00012665863), (26, 0.00012665863), (27, 0.00012665863), (28, 0.00012665863), (29, 0.00012665863), (30, 0.00012665863), (31, 0.0838537), (32, 0.00012665863), (33, 0.00012665863), (34, 0.00012665863), (35, 0.00012665863), (36, 0.00012665863), (37, 0.011350646), (38, 0.00012665863), (39, 0.00012665863), (40, 0.00012665863), (41, 0.13744669), (42, 0.00012665863), (43, 0.00012665863), (44, 0.00012665863), (45, 0.079688594), (46, 0.015319362), (47, 0.00012665863), (48, 0.052460793), (49

In [34]:
with open('doc_lda_tensor.tsv','w') as w:
    for doc_topics in all_topics:
        for topics in doc_topics:
            w.write(str(topics[1])+ "\t")
        w.write("\n")    

In [35]:
# meta = [str(a).split("/") for a in articles]
with open('doc_lda_metadata.tsv','w') as w:
    w.write('Titles\tGenres\n')
    for m in df_nonna['id']:
        w.write("%s\n" % (m))
#     for m in meta:
#         w.write("%s\t%s\n" % (m[1].split("-")[0], m[1]))

In [39]:
tags_str = df_nonna["lang"]

lang_s = []
for lang in tags_str:
#     if(len(lang_s) > 50):
#         break
    lang_s.append(lang.split(",")[0])


In [41]:
with open('doc_lda_metadata.tsv','w') as w:
    w.write('Titles\tGenres\n')
    for m in lang_s:
        w.write("%s\t%s\n" % (m, "none"))

In [42]:
len(df_nonna)

2790